In [7]:
# imports
from tkinter import *
import os
from PIL import ImageTk, Image
import logging
logging.basicConfig(filename = 'withdraw.log', level = logging.DEBUG, format = '%(asctime)s %(levelname)s %(message)s' )

# Main Screen
master = Tk()
master.title('Banking App')

# Functions
def finish_reg():
    name = temp_name.get()
    age = temp_age.get()
    gender = temp_gender.get()
    password = temp_password.get()
    all_accounts = os.listdir()
    
    try:
        if name == "" or age == "" or gender == "" or password == "":
            notify.config(fg='red',text='All fields required')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
        
    try:
        for name_check in all_accounts:
            if name == name_check:
                notify.config(fg='red',text='Account already exists')
                return
            else:
                new_file = open(name, 'w')
                new_file.write(name+'\n')
                new_file.write(age+'\n')
                new_file.write(gender+'\n')
                new_file.write(password+'\n')
                new_file.write('0')
                new_file.close()
                notify.config(fg='green', text='Account has been created')
    except Exception as e:
        logging.exception('Exception occured '+str(e))

def register():
    # Vars
    global temp_name
    global temp_age
    global temp_gender
    global temp_password
    global notify
    temp_name = StringVar()
    temp_age = StringVar()
    temp_gender = StringVar()
    temp_password = StringVar()
    
    # Register Screen
    register_screen = Toplevel(master)
    register_screen.title('Register')
    
    # Labels
    Label(register_screen, text='Please enter your details below to register',font=('Calibri',12)).grid(row=0,sticky=N,pady=10)
    Label(register_screen, text='Name',font=('Calibri',12)).grid(row=1,sticky=W)
    Label(register_screen, text='Age',font=('Calibri',12)).grid(row=2,sticky=W)
    Label(register_screen, text='Gender',font=('Calibri',12)).grid(row=3,sticky=W)
    Label(register_screen, text='Password',font=('Calibri',12)).grid(row=4,sticky=W)
    notify = Label(register_screen, font=('Calibri',12))
    notify.grid(row=6,sticky=N, pady=10)
    
    # Entries
    Entry(register_screen,textvariable=temp_name).grid(row=1,column=0)
    Entry(register_screen,textvariable=temp_age).grid(row=2,column=0)
    Entry(register_screen,textvariable=temp_gender).grid(row=3,column=0)
    Entry(register_screen,textvariable=temp_password,show='*').grid(row=4,column=0)
    
    # Buttons
    Button(register_screen, text='Register', command=finish_reg, font=('Calibri',12)).grid(row=5,sticky=N,pady=10)

def login_session():
    global login_name
    all_accounts = os.listdir()
    login_name = temp_login_name.get()
    login_password = temp_login_password.get()
    try:
        for name in all_accounts:
            if name == login_name:
                file = open(name, 'r')
                file_data = file.read()
                file_data = file_data.split('\n')
                password = file_data[3]
                
                try:
                    # Account Dashboard
                    if login_password == password:
                        login_screen.destroy()
                        account_dashboard = Toplevel(master)
                        account_dashboard.title('Dashboard')
                        # labels
                        Label(account_dashboard, text='Account Dashboard', font=('Calibri',12)).grid(row=0,sticky=N,pady=10)
                        Label(account_dashboard, text='Welcome '+name, font=('Calibri',12)).grid(row=1,sticky=N,pady=5)
                        # Buttons
                        Button(account_dashboard,text='Personal Details',font=('Calibri',12),width=30,command=personal_details).grid(row=2,sticky=N,padx=10)
                        Button(account_dashboard,text='Deposit',font=('Calibri',12),width=30,command=deposit).grid(row=3,sticky=N,padx=10)
                        Button(account_dashboard,text='Withdraw',font=('Calibri',12),width=30,command=withdraw).grid(row=4,sticky=N,padx=10)
                        Label(account_dashboard).grid(row=5,sticky=N,pady=10)
                        return
                    else:
                        login_notify.config(fg='red',text='Password incorrect!!')
                        return
                except Exception as e:
                    logging.exception('Exception occured '+str(e))
                    
    except Exception as e:
        logging.exception('Exception occured '+str(e))
        
    login_notify.config(fg='red', text='No Account Fond !!')

def deposit():
    # Vars
    global amount
    global deposit_notify
    global current_balance_label
    
    try:
        amount = StringVar()
        file = open(login_name, 'r')
        file_data = file.read()
        user_details = file_data.split('\n')
        details_balance = user_details[4]
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    # Deposit Screen
    deposit_screen = Toplevel(master)
    deposit_screen.title('Deposit')
    
    # Label
    Label(deposit_screen, text='Deposit', font=('Calibri',12)).grid(row=0,sticky=N,pady=10)
    current_balance_label = Label(deposit_screen, text='Current balance : $ '+details_balance, font=('Calibri',12))
    current_balance_label.grid(row=1,sticky=W)
    Label(deposit_screen, text='Amount', font=('Calibri',12)).grid(row=2,sticky=W)
    deposit_notify = Label(deposit_screen, font=('Calibri',12))
    deposit_notify.grid(row=4,sticky=N,pady=5)
    
    # Entry
    Entry(deposit_screen, textvariable=amount).grid(row=2,column=1)
    
    # Button
    Button(deposit_screen, text='Finish',font=('Calibri',12),command=finish_deposit).grid(row=3,sticky=W,pady=5)

def finish_deposit():
    try:
        if amount.get() == "":
            deposit_notify.config(text='Amount is required',fg='red')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    try:
        if float(amount.get()) <= 0:
            deposit_notify.config(text='Negative currency is not accepted',fg='red')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    try:
        file = open(login_name, 'r+')
        file_data = file.read()
        details = file_data.split('\n')
        current_balance = details[4]
        updated_balance = current_balance
        updated_balance = float(updated_balance) + float(amount.get())
        file_data = file_data.replace(current_balance, str(updated_balance))
        file.seek(0)
        file.truncate(0)
        file.write(file_data)
        file.close()
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    current_balance_label.config(text='Current Balance : $ '+str(updated_balance),fg='green')
    deposit_notify.config(text='Balance Updated',fg='green')
    
def withdraw():
    # Vars
    global withdraw_amount
    global withdraw_notify
    global current_balance_label
    
    try:
        withdraw_amount = StringVar()
        file = open(login_name, 'r')
        file_data = file.read()
        user_details = file_data.split('\n')
        details_balance = user_details[4]
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    # Deposit Screen
    withdraw_screen = Toplevel(master)
    withdraw_screen.title('Withdraw')
    
    # Label
    Label(withdraw_screen, text='Withdraw', font=('Calibri',12)).grid(row=0,sticky=N,pady=10)
    current_balance_label = Label(withdraw_screen, text='Current balance : $ '+details_balance, font=('Calibri',12))
    current_balance_label.grid(row=1,sticky=W)
    Label(withdraw_screen, text='Amount', font=('Calibri',12)).grid(row=2,sticky=W)
    withdraw_notify = Label(withdraw_screen, font=('Calibri',12))
    withdraw_notify.grid(row=4,sticky=N,pady=5)
    
    # Entry
    Entry(withdraw_screen, textvariable=withdraw_amount).grid(row=2,column=1)
    
    # Button
    Button(withdraw_screen, text='Finish',font=('Calibri',12),command=finish_withdraw).grid(row=3,sticky=W,pady=5)

def finish_withdraw():
    try:
        if withdraw_amount.get() == "":
            withdraw_notify.config(text='Amount is required',fg='red')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
        
    try:
        if float(withdraw_amount.get()) <= 0:
            withdraw_notify.config(text='Negative currency is not accepted',fg='red')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
        withdraw_notify.config(text=e,fg='red')
    
    try:
        file = open(login_name, 'r+')
        file_data = file.read()
        details = file_data.split('\n')
        current_balance = details[4]
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    try:
        if float(withdraw_amount.get()) > float(current_balance):
            withdraw_notify.config(text='Insufficent Funds!',fg='red')
            return
    except Exception as e:
        logging.exception('Exception occured '+str(e))
        withdraw_notify.config(text=e,fg='red')
    
    try:
        updated_balance = current_balance
        updated_balance = float(updated_balance) - float(withdraw_amount.get())
        file_data = file_data.replace(current_balance, str(updated_balance))
        file.seek(0)
        file.truncate(0)
        file.write(file_data)
        file.close()
    
        current_balance_label.config(text='Current Balance : $ '+str(updated_balance),fg='green')
        withdraw_notify.config(text='Balance Updated',fg='green')
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
def personal_details():
    try:
        # Vars
        file = open(login_name, 'r')
        file_data = file.read()
        user_details = file_data.split('\n')
        details_name = user_details[0]
        details_age = user_details[1]
        details_gender = user_details[2]
        details_balance = user_details[4]
    except Exception as e:
        logging.exception('Exception occured '+str(e))
    
    # Persoanl Details Screen
    personal_details_screen = Toplevel(master)
    personal_details_screen.title('Personal Details')
    
    # Labels
    Label(personal_details_screen, text='Personal Details', font=('Calibri',12)).grid(row=0,sticky=N,pady=10)
    Label(personal_details_screen, text='Name : '+details_name, font=('Calibri',12)).grid(row=1,sticky=W)
    Label(personal_details_screen, text='Age : '+details_age, font=('Calibri',12)).grid(row=2,sticky=W)
    Label(personal_details_screen, text='Gender : '+details_gender, font=('Calibri',12)).grid(row=3,sticky=W)
    Label(personal_details_screen, text='Balance : $ '+details_balance, font=('Calibri',12)).grid(row=4,sticky=W)
    
def login():
    # Vars
    global temp_login_name
    global temp_login_password
    global login_notify
    global login_screen
    
    temp_login_name = StringVar()
    temp_login_password = StringVar()
    
    # Login Screen
    login_screen = Toplevel(master)
    login_screen.title('Login')
    
    # Labels
    Label(login_screen, text = 'Login to your account', font = ('Calibri',12)).grid(row=0,sticky=N,pady=10)
    Label(login_screen, text = 'Username', font = ('Calibri',12)).grid(row=1,sticky=W)
    Label(login_screen, text = 'Password', font = ('Calibri',12)).grid(row=2,sticky=W)
    login_notify = Label(login_screen, font=('Calibri',12))
    login_notify.grid(row=4,stick=N)
    
    # Entry
    Entry(login_screen, textvariable=temp_login_name).grid(row=1,column=1,padx=5)
    Entry(login_screen, textvariable=temp_login_password,show='*').grid(row=2,column=1,padx=5)
    
    # Button
    Button(login_screen, text='Login', command=login_session, width=15, font=('Calibri',12)).grid(row=3,sticky=W,pady=5,padx=5)

# Image import
img = Image.open('bank.png')
img = img.resize((150,150))
img = ImageTk.PhotoImage(img) 

# Labels
Label(master, text = 'Custom Banking Beta', font = ('Calibri', 14)).grid(row=0,sticky=N,pady=10)
Label(master, text = 'The Most Trusted Bank', font = ('Calibri', 12)).grid(row=1,sticky=N)
Label(master, image=img).grid(row=2,sticky=N,pady=15)

# Buttons
Button(master, text = 'Register', font=('Calibri', 12), width = 20, command=register).grid(row=3,sticky=N)
Button(master, text = 'Login', font=('Calibri', 12), width = 20,command=login).grid(row=4,sticky=N,pady=5)

master.mainloop()